# RAVDESS
Calculate audio features from dataset and export to dataframe for modeling in another notebook.

In [1]:
import librosa
import librosa.display
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from sklearn import preprocessing

In [2]:
import  os

# Data Processing

## Load and Label Data Set

In [3]:
# read in data files

files = sorted(os.listdir('data/RAVDESS/'))

In [4]:
# apply labels based on filename coding

emotion_labels = []

for file in files:
    
    if file[6:8]=='01' and int(file[18:20])%2==0:
        emotion_labels.append('neutral')
    elif file[6:8]=='01' and int(file[18:20])%2==1:
        emotion_labels.append('neutral')
    elif file[6:8]=='02' and int(file[18:20])%2==0:
        emotion_labels.append('calm')
    elif file[6:8]=='02' and int(file[18:20])%2==1:
        emotion_labels.append('calm')
    elif file[6:8]=='03' and int(file[18:20])%2==0:
        emotion_labels.append('happy')
    elif file[6:8]=='03' and int(file[18:20])%2==1:
        emotion_labels.append('happy')
    elif file[6:8]=='04' and int(file[18:20])%2==0:
        emotion_labels.append('sad')
    elif file[6:8]=='04' and int(file[18:20])%2==1:
        emotion_labels.append('sad')
    elif file[6:8]=='05' and int(file[18:20])%2==0:
        emotion_labels.append('angry')
    elif file[6:8]=='05' and int(file[18:20])%2==1:
        emotion_labels.append('angry')
    elif file[6:8]=='06' and int(file[18:20])%2==0:
        emotion_labels.append('fearful')
    elif file[6:8]=='06' and int(file[18:20])%2==1:
        emotion_labels.append('fearful')
    elif file[6:8]=='07' and int(file[18:20])%2==0:
        emotion_labels.append('disgust')
    elif file[6:8]=='07' and int(file[18:20])%2==1:
        emotion_labels.append('disgust')
    elif file[6:8]=='08' and int(file[18:20])%2==0:
        emotion_labels.append('surprised')
    elif file[6:8]=='08' and int(file[18:20])%2==1:
        emotion_labels.append('surprised')
    else:
        emotion_labels.append('unlabeled')

In [5]:
labels = pd.DataFrame(emotion_labels, columns=['emotion_label'])

In [6]:
labels['file'] = pd.Series(files)

## Calculate Features
Analysis uses Mel Frequency Cepstral Coefficients (MFCC) to calculate features.

In [7]:
df = pd.DataFrame(columns=['feature'])

In [8]:
# set librosa parametrs
offset = 0.5 # time (s) to offset audio file start
duration = 2.5 # selected duration for each file (s)
sr = 22050 # sample rate (Hz)
n_mfcc = 13 # number of cepstral coefs to return
frame_size = 0.025 # window size (s)
n_fft = int(frame_size*sr) # number of frames
frame_stride = 0.01 # stride size (s)
hop_length = int(frame_stride*sr) # number of samples to hop per frame

In [10]:
# calculate mfcc features for each file
for index, file in enumerate(files[0:2]):
    
    y, sr = librosa.load('data/RAVDESS/' + file, duration=duration, sr=sr, offset=offset) # return sampled file data
    if len(y) < duration*sr: 
        y = np.pad(y, (0, int(duration*sr - len(y))), 'constant') # for files < duration, pad with zero values
    # return matrix of # mfccs x # frames
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc, n_fft=n_fft, hop_length=hop_length) 
    mfccs = preprocessing.scale(mfccs, axis=1) # scale coefficents
    feature = mfccs.reshape(mfccs.shape[0] * mfccs.shape[1]) # reshape mfcc matrix into 1-D array
    
    df.loc[index] = [feature] # place feature array into dataframe

In [16]:
mfccs.shape

(13, 251)

In [11]:
# create dataframe
feat_df = pd.DataFrame(df['feature'].values.tolist())

In [12]:
# add emotion lables
feat_df = pd.concat([feat_df, labels.loc[:, 'emotion_label']], axis=1)

In [13]:
feat_df.head()

,0,1,2,3,4,5,6,7,8,9,...,3254,3255,3256,3257,3258,3259,3260,3261,3262,emotion_label
0,-1.105108,-1.105108,-1.105108,-1.105108,-1.105108,-1.105108,-1.105108,-1.105108,-1.105108,-1.105108,...,-0.238322,-0.279292,-0.215209,-0.141943,-0.265668,-0.259120,-0.004098,-0.261109,-0.132365,neutral
1,-1.370970,-1.370970,-1.370970,-1.370970,-1.370970,-1.370970,-1.370970,-1.288133,-1.278144,-1.366667,...,0.188097,0.191922,0.896465,1.110646,1.218238,1.219607,0.731544,0.909255,1.093390,neutral
2,-1.328020,-1.328020,-1.328020,-1.328020,-1.328020,-1.328020,-1.328020,-1.328020,-1.328020,-1.328020,...,1.049234,0.867329,1.333470,1.060284,0.908868,1.061795,1.752833,0.271956,0.335296,neutral
3,-1.169856,-1.169856,-1.169856,-1.169856,-1.169856,-1.169856,-1.169856,-1.169856,-1.169856,-1.169856,...,0.659684,0.180330,0.673990,0.463311,0.469228,0.341113,0.554891,0.596502,0.689033,neutral
4,-1.438135,-1.412139,-1.488253,-1.490683,-1.490683,-1.429146,-1.424162,-1.490102,-1.475640,-1.430225,...,0.647851,1.035051,0.521052,0.828227,-0.113310,0.352832,-0.114662,0.283366,0.342459,neutral


## Export data

In [14]:
import pickle

In [15]:
pickle_out = open("ravdess.pickle", "wb")
pickle.dump(feat_df, pickle_out)
pickle_out.close()